In [ ]:
# turjuman
! pip install git+https://github.com/UBC-NLP/turjuman.git --q

  Preparing metadata (setup.py) ... done


In [ ]:
from tqdm.auto import tqdm
import json
import numpy as np
import re
import logging
import os
from turjuman import turjuman
import random
import numpy as np
from nltk.corpus import stopwords
from collections import OrderedDict

In [ ]:
! wget --no-clobber https://raw.githubusercontent.com/essawey/Marare_project/main/src/JSON%20Data/ID_TO_CAPTION_EN.json

File ‘ID_TO_CAPTION_EN.json’ already there; not retrieving.



In [ ]:
with open('ID_TO_CAPTION_EN.json', 'r') as f:
    ID_TO_CAPTION_EN = json.load(f,object_pairs_hook=OrderedDict)

In [ ]:
def DICT_TO_TEXT(dict, path):
  with open(path, "w") as f:
      for value in dict.values():
          for item in value:
              if item.endswith("\n"):
                  item = item[:-1]  # remove the newline
              f.write(item + "\n")

In [ ]:
DICT_TO_TEXT(dict = ID_TO_CAPTION_EN , path = "CATIONS_EN.txt")

In [ ]:
# Initial turjuman object
logging.basicConfig(
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=os.environ.get("LOGLEVEL", "INFO").upper(),
)
logger = logging.getLogger("turjuman.translate")
cache_dir="/content/mycache"

torj = turjuman.turjuman(logger, cache_dir)
beam_options = {"search_method":"beam", "seq_length": 300, "num_beams":5, "no_repeat_ngram_size":2, "max_outputs":1}

In [ ]:
def get_var_name(var):
    for name in globals():
        if globals()[name] is var:
            return name
    for name in locals():
        if locals()[name] is var:
            return name
    return None

In [ ]:
def split_file(input_file, num_files, output_dir):
    """
    Splits a large text file into smaller files and saves them to a specified output directory.

    Parameters:
    input_file (str): The path of the input file to be split.
    num_files (int): The number of output files that the input file should be split into.
    output_dir (str): The path of the directory where the output files will be saved.

    Returns:
    None

    Example Usage:
    split_file("input_file_path", 3, "output_directory_path")
    """
    # Read the lines of the input file into a list
    with open(input_file+".txt", 'r') as f:
        lines = f.readlines()

    # Split the lines into N smaller arrays
    arrays = np.array_split(lines, num_files)

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Write each smaller array to a separate file
    for i, array in enumerate(arrays):
        # Open a new output file for each array and write its lines
        with open(f"{output_dir}/{input_file}_{i+1}.txt", 'w') as f:
            for j, line in enumerate(array):
                if j == len(array) - 1:
                    # Remove the newline character from the last line
                    line = line.rstrip('\n')
                if line.strip():  # Check if the line is not empty
                    f.write(line)

    # Print a message to indicate how many files were successfully split
    print(f"{num_files} files split successfully.")

In [ ]:
def merge_files(input_folder_path, output_file_path):
    """
    Merges the contents of multiple files in a specified folder into a single output file.

    Parameters:
    input_folder_path (str): The path of the folder containing the input files.
    output_file_path (str): The path of the output file that will contain the merged contents.

    Returns:
    None

    Example Usage:
    merge_files("input_folder_path", "output_file_path")
    """
    
    # Get list of files in specified folder
    file_list = []
    for filename in os.listdir(input_folder_path):
        # If file is not a directory, add its path to the file list
        if os.path.isfile(os.path.join(input_folder_path, filename)):
            filename = input_folder_path + "/" + filename
            file_list.append(filename)

    # Sort the file list based on the numerical value found in the file names
    file_paths_list = sorted(file_list, key=lambda x: int(re.search(r'\d+', x).group()))

    # Merge the contents of the files into a single output file
    with open(output_file_path, "w", encoding='utf-8') as output_file:
        for file_path in file_paths_list:
            # Open each file in read mode and write its contents to the output file
            with open(file_path, "r", encoding='utf-8') as input_file:
                output_file.write(input_file.read() + "\n")

    # Print a message to indicate how many files were successfully merged
    print(f"{len(os.listdir(input_folder_path))} files merged successfully")

In [ ]:
output_dir = "CATIONS_AR_dir"
folder_path = f"/content/{output_dir}"
split_file(input_file = "CATIONS_EN" , num_files = 10, output_dir = output_dir)
merge_files(input_folder_path = folder_path , output_file_path = 'CATIONS_AR.txt')

10000 files split successfully.
10000 files merged successfully


<div class="alert alert-success" role="alert">
      <h4
          class="alert-heading">Importing Libraries
      </h4>  
</div>


In [ ]:
import os
import re
import string

# NLTK
from nltk.corpus import stopwords
from nltk import ngrams

# similarly - Word2Vec
import gensim

In [ ]:
torj.translate_from_file("/content/CATIONS_AR_dir/CATIONS_EN_1.txt", batch_size = 25, **beam_options)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating source split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /content/mycache/text/default-eec8df1ab7e92f88/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]


translate: 100%|██████████| 3/3 [01:20<00:00, 26.90s/it]


In [ ]:
import pandas as pd
df = pd.read_json("/content/CATIONS_AR_dir/CATIONS_EN_1_Turjuman_translate.json", orient='records', lines=True)
df['target'].to_csv('CAPTION_AR.txt', index=False, sep='\n')

In [ ]:
def remove_first_line(file_path):
    # Open the file for reading
    with open(file_path, 'r', encoding='utf-8') as f:
        # Read all lines from the file into a list
        lines = f.readlines()
    # Remove the first line from the list
    lines.pop(0)
    # Open the file for writing
    with open(file_path, 'w', encoding='utf-8') as f:
        # Write the remaining lines back to the file
        f.writelines(lines)

In [ ]:
remove_first_line("CAPTION_AR.txt")